In [1]:
%%file  sijuntao_si618_hw5_part1.py

import mrjob
import nltk
import re
from mrjob.job import MRJob
from nltk.stem.porter import *

pattern = r'\b[A-Za-z]{4,}\b'
stemmer = PorterStemmer()

class MRMostUsedWordStems(MRJob):
    OUTPUT_PROTOCOL = mrjob.protocol.TextProtocol

    def mapper(self, _, line):
        try:
            congress = line.split('\t')[4]  # congress
            if line != '' and line is not None:
                origin_word_list = re.findall(pattern, line)
                for origin_word in origin_word_list:
                    stem_word = stemmer.stem(origin_word)
                    yield (congress, stem_word), 1
        except:
            pass
       
    def combiner(self, word_stem, counts):
        # sums up the count for each (congress, stem) pair
        yield word_stem, sum(counts)

    def reducer(self, word_stem, counts):
        # gets the final list of counts for each (congress, word_stem)
        yield  word_stem[0] + "\t" + word_stem[1], str(sum(counts))
        

if __name__ == "__main__":
    MRMostUsedWordStems.run()


Overwriting sijuntao_si618_hw5_part1.py


In [2]:
!python3 sijuntao_si618_hw5_part1.py ./bills -o ./output_part1

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /var/folders/10/lx3rh6q956qc768glw5ktj600000gn/T/sijuntao_si618_hw5_part1.junsi.20221001.170609.590628
job output is in ./output_part1
Removing temp directory /var/folders/10/lx3rh6q956qc768glw5ktj600000gn/T/sijuntao_si618_hw5_part1.junsi.20221001.170609.590628...


In [3]:
!cat ./output_part1/part* > sijuntao_si618_hw5_output_part1.txt

In [4]:
!diff sijuntao_si618_hw5_output_part1.txt si618_hw4_part1_desired_output.txt

In [5]:
%%file  sijuntao_si618_hw5_part2.py

import mrjob
import nltk
import re
from mrjob.job import MRJob
from mrjob.step import MRStep
from nltk.stem.porter import *

pattern = r'\b[A-Za-z]{4,}\b'
stemmer = PorterStemmer()

class MRMostUsedWordStems(MRJob):
    OUTPUT_PROTOCOL = mrjob.protocol.TextProtocol

    def mapper_get_word_stems(self, _, line):
        try:
            if line != '' and line is not None:
                origin_word_list = re.findall(pattern, line)
                for origin_word in origin_word_list:
                    stem_word = stemmer.stem(origin_word)
                    yield stem_word, 1
        except:
            pass
       
    def combiner_count_word_stems(self, word_stem, counts):
        # sums up the count for each stem
        yield word_stem, sum(counts)

    def reducer_count_word_stems(self, word_stem, counts):
        # gets the tuple of (stem_length, (word_stem, counts))
        yield  len(word_stem), (word_stem, sum(counts))
        
    def reducer_find_max_word_stem(self, length, word_stem_count_pairs):
        # gets most frequent word stem for each word stem length
        max_word_stem_count_pairs = max(word_stem_count_pairs, key = lambda pair: list(pair)[1])
        yield  str(length) + '\t' + list(max_word_stem_count_pairs)[0], str(list(max_word_stem_count_pairs)[1])
    
    def steps(self):
        return [
            MRStep(mapper = self.mapper_get_word_stems,
                   combiner = self.combiner_count_word_stems,
                   reducer = self.reducer_count_word_stems),
            MRStep(reducer = self.reducer_find_max_word_stem)
        ]
        

if __name__ == "__main__":
    MRMostUsedWordStems.run()


Overwriting sijuntao_si618_hw5_part2.py


In [6]:
!python3 sijuntao_si618_hw5_part2.py ./bills -o ./output_part2

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 2...
Creating temp directory /var/folders/10/lx3rh6q956qc768glw5ktj600000gn/T/sijuntao_si618_hw5_part2.junsi.20221001.170907.394330
Running step 2 of 2...
job output is in ./output_part2
Removing temp directory /var/folders/10/lx3rh6q956qc768glw5ktj600000gn/T/sijuntao_si618_hw5_part2.junsi.20221001.170907.394330...


In [7]:
!cat ./output_part2/part* > sijuntao_si618_hw5_output_part2.txt

In [8]:
!diff sijuntao_si618_hw5_output_part2.txt si618_hw4_part2_desired_output.txt